# 0. Setup

Load the modules

In [ ]:
from seal import *

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import tensorflow as tf
import random
from sklearn import svm
import requests
import tensorflow
from keras.models import Sequential
import warnings
from sklearn.preprocessing import normalize
warnings.filterwarnings(action='ignore')

%matplotlib inline

# 1. Key Generation
- Define the key path
- If there is no generated key pair, use this code to generate key set.

In [ ]:
PK_PATH = 'keyset/public_key' # 'Define the pbulic key path'
SK_PATH = 'keyset/secret_key' # 'Define the secret key path'
GK_PATH = 'keyset/galois_keys' # 'Define the galois key path'
RL_PATH = 'keyset/relin_key' # 'Define the relinearization key path'

# 2. CKKS scheme's parameter setting (In the SEAL-Python)

In [ ]:
parms = EncryptionParameters(scheme_type.ckks)
poly_modulus_degree = 16384
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.Create(
        poly_modulus_degree, [60, 40, 40, 40, 60]))
scale = 2.0**40
context = SEALContext(parms) 
ckks_encoder = CKKSEncoder(context)
slot_count = ckks_encoder.slot_count()
print(f'Number of slots: {slot_count}')
 

public_key = PublicKey()
public_key.load(context, PK_PATH)
secret_key = SecretKey()
secret_key.load(context, SK_PATH) 
galois_key = GaloisKeys()
galois_key.load(context, GK_PATH)
relin_keys = RelinKeys()
relin_keys.load(context, RL_PATH)

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

# 3. Load dataset
- Load the test dataset which is saved in the training

In [ ]:
TESTSET_PATH = 'The test set of the sokoto dataset from the training'
test_data = np.load(TESTSET_PATH, allow_pickle=True)
test_data.shape

# 4. Load models

- Load the models which are saved in the training

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

swish = tensorflow.keras.activations.swish

custom_objects={"f1_m": f1_m, "precision_m":precision_m, "recall_m":recall_m}

def sigmoid(x):
    return 1 / (1 +np.exp(-x))

In [ ]:
FEATURE_MODEL_PATH = 'Feature model path defined in the training'
MODEL_PATH = 'Model path defined in the training'
 
feature_model = tf.keras.models.load_model(FEATURE_MODEL_PATH, custom_objects=custom_objects)
model = tf.keras.models.load_model(MODEL_PATH, custom_objects=custom_objects)

# 5. For Pre-processing (defined in our paper)
- Adjust the parameter and do test.

In [ ]:
test_seq = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, 0.7)),
    iaa.Dropout((0.01, 0.15), per_channel=0.5),
    iaa.Affine(
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        rotate=(-30, 30),
        order=[0, 1],
        cval=1
    )
], random_order=True)

# 6. Split the 1,200 test set to 3 part (For three clusters)

## 6-1. Extract the feature vectors

In [ ]:
weight = model.weights[30]
bias = model.weights[31]

result1 = feature_model.predict(test_data[:512])
result1 = keras.layers.UnitNormalization()(layers.Flatten()(result1))
result1 = np.matmul(result1, weight)

result2 = feature_model.predict(test_data[512:1024])
result2 = keras.layers.UnitNormalization()(layers.Flatten()(result2))
result2 = np.matmul(result2, weight)

result3 = feature_model.predict(test_data[1024:])
result3 = keras.layers.UnitNormalization()(layers.Flatten()(result3))
result3 = np.matmul(result3, weight)

## 6-2 Encrypt the three sets of the feature vectors 

In [ ]:
ctxt1 = encryptor.encrypt(ckks_encoder.encode(result1.flatten(), scale))
ctxt2 = encryptor.encrypt(ckks_encoder.encode(result2.flatten(), scale))
ctxt3 = encryptor.encrypt(ckks_encoder.encode(result3.flatten(), scale))

In [ ]:
CTXT1_PATH = 'Path of the ctxt1'
CTXT2_PATH = 'Path of the ctxt2'
CTXT3_PATH = 'Path of the ctxt3'

ctxt1.save(CTXT1_PATH)
ctxt2.save(CTXT2_PATH)
ctxt3.save(CTXT3_PATH)

## TEST

In [ ]:
from time import time
MAX_FINGER_NUM = 512
THRESHOLD = 0.99 # Set your threshold
URL = 'MAIN_SERVER_IP:MAIN_SERVER_PORT/blindtouch' # The URL of the main server

idx = random.randint(0, 1199)
print('randomly chosen idx : ', idx)
test = test_seq.augment_image(test_data[idx]) # Pre-processing

START_TIME = time()
scaled_x_test = [] 

predicted_ = feature_model.predict(test.reshape([1,224,224,1]))
predicted_ = keras.layers.UnitNormalization()(layers.Flatten()(predicted_))
predicted = np.matmul(predicted_, weight) + bias
predicted = np.array(predicted.numpy().tolist()[0]* MAX_FINGER_NUM)


ctxt = encryptor.encrypt(ckks_encoder.encode(predicted, scale))
ctxt.save('temp_ctxt')
files = open('temp_ctxt', 'rb')
upload = {'target_enc': files}
response = requests.post(URL, files = upload) 

with open('./result_enc', mode='wb') as localfile:     
    localfile.write(response.content) 
FINISH_TIME = time()
ctxt2 = Ciphertext()
ctxt2.load(context, './result_enc')
    
vec = ckks_encoder.decode(decryptor.decrypt(ctxt2))
print('TOTAL TIME (s): ', time()-START_TIME) 
result_list = []
for i in range(3):
    for j in range(512):
        result_list.append(sigmoid(vec[j * 16 + i]))

for ii in range(len(result_list)):
    if result_list[ii] > THRESHOLD :
        print('Predicted idx: ', ii)